# OpenAI Assistant V2 - RAG 테스트

- 저작자: [테디노트](https://www.youtube.com/@teddynote)

**코드 재배포시 출처를 남겨 주세요**

In [ ]:
# 라이브러리 업데이트
!pip install -qU langchain-teddynote

**OpenAI API KEY 발급 방법**

- [OpenAI 키 발급방법](https://wikidocs.net/233342)

In [1]:
# 발급 받은 키를 입력해 주세요
openai_api_key = "OPENAI API KEY 입력"

`.env` 에 OpenAI API KEY 를 저장해 두었다면 밑의 코드 주석을 해제하고 실행하세요.

In [ ]:
# from dotenv import load_dotenv
# import os

# load_dotenv()

# openai_api_key = os.getenv("OPENAI_API_KEY")

## 주요 링크

- [OpenAI Assistant 대시보드](https://platform.openai.com/assistants)
- [OpenAI API KEY 발급](https://platform.openai.com/api-keys)
- [OpenAI Files 스토리지](https://platform.openai.com/storage/files)

## 실습에 활용한 문서

소프트웨어정책연구소(SPRi) - 2023년 12월호

- 저자: 유재흥(AI정책연구실 책임연구원), 이지수(AI정책연구실 위촉연구원)
- 링크: https://spri.kr/posts/view/23669
- 파일명: `SPRI_AI_Brief_2023년12월호_F.pdf`

## STEP 1) 설정

- `_DEFAULT_RAG_INSTRUCTIONS` 는 RAG 의 instruction 입니다. 프롬프트를 자유롭게 수정하여 테스트해 보세요.
- `OPENAI_API_KEY` 는 발급받은 OPENAI API KEY 를 입력해 주세요.
- `PROJECT_NAME` 은 프로젝트 이름을 입력해 주세요.
- `model_name` 은 사용할 모델을 입력해 주세요. (예, `gpt-4o`, `gpt-4o-mini`, ...)


In [2]:
# RAG 시스템 프롬프트 입력
_DEFAULT_RAG_INSTRUCTIONS = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean."""

# 설정(configs)
configs = {
    "OPENAI_API_KEY": openai_api_key,  # OpenAI API 키
    "instructions": _DEFAULT_RAG_INSTRUCTIONS,  # RAG 시스템 프롬프트
    "PROJECT_NAME": "PDF-RAG-TEST",  # 프로젝트 이름(자유롭게 설정)
    "model_name": "gpt-4o",  # 사용할 OpenAI 모델 이름(gpt-4o, gpt-4o-mini, ...)
}

In [3]:
from langchain_teddynote.models import OpenAIAssistant

# 인스턴스 생성
assistant = OpenAIAssistant(configs)

## STEP 2) 파일 업로드

### 파일 업로드 하지 않은 경우

- RAG 할 파일을 업로드 합니다.
- 파일 업로드는 딱 1번만 하면 됩니다.

이미 파일을 업로드 하였고, file_id 가 있다면, 아래 코드는 생략합니다.

In [4]:
# 업로드할 파일 경로
data = "SPRI_AI_Brief_2023년12월호_F.pdf"

# 파일 업로드 후 file_id 는 잘 보관해 두세요. (대시보드에서 나중에 확인 가능)
file_id = assistant.upload_file(data)

## STEP 3-1) Assistant, VectorStore 설정

(참고) 이미 있는 assistant_id, vector_id 가 있다면, `STEP 3-2` 으로 넘어가세요.

- `assistant_id`, `vector_id` 는 잘 보관해 두세요 (메모장 같은 곳에...)

In [6]:
# 업로드한 파일의 ID 리스트 생성
file_ids = [file_id]

# 새로운 어시스턴트 생성 및 ID 받기
assistant_id, vector_id = assistant.create_new_assistant(file_ids)

# 어시스턴트 설정
assistant.setup_assistant(assistant_id)

# 벡터 스토어 설정
assistant.setup_vectorstore(vector_id)

## STEP 3-2) 이미 있는 assistant_id, vector_id 가 있다면, 아래와 같이 설정합니다.

만약에 이미 있는 assistant_id, vector_id 가 있다면, 아래와 같이 설정합니다.

In [ ]:
# assistant_id = "asst_465H6MZzEs6yabcsdfdfdl"
# vector_id = "vs_jVBYPLHVrxTvQsdfsgsfsf"

# # 어시스턴트 설정
# assistant.setup_assistant(assistant_id)

# # 벡터 스토어 설정
# assistant.setup_vectorstore(vector_id)

## STEP 4) 대화

In [7]:
# 질문
print(assistant.invoke("삼성전자가 개발한 생성형 AI의 이름은?"))

삼성전자가 개발한 생성형 AI의 이름은 '삼성 가우스'입니다 .


In [8]:
# 추가 질문
print(assistant.invoke("이전의 답변에 대한 출처(페이지)를 알려줘"))

삼성전자가 개발한 생성형 AI의 이름은 '삼성 가우스'입니다【8:0†source】.


In [9]:
# 대화 목록 조회
assistant.list_chat_history()

[{'content': '삼성전자가 개발한 생성형 AI의 이름은?', 'role': 'user'},
 {'content': "삼성전자가 개발한 생성형 AI의 이름은 '삼성 가우스'입니다 .", 'role': 'assistant'},
 {'content': '이전의 답변에 대한 출처(페이지)를 알려줘', 'role': 'user'},
 {'content': "삼성전자가 개발한 생성형 AI의 이름은 '삼성 가우스'입니다【8:0†source】.",
  'role': 'assistant'}]

새로운 주제는 새로운 대화 쓰레드(thread)를 시작해야 합니다.

In [10]:
# 대화 초기화
assistant.clear_chat_history()

In [11]:
# 질문
print(assistant.invoke("구글이 앤쓰로픽에 투자 계획을 발표했습니다. 이에 대해 알려줘"))

구글은 2023년 10월 27일에 앤쓰로픽에 최대 20억 달러를 투자하기로 합의했습니다. 이 중 5억 달러를 우선 투자하였으며, 나머지 15억 달러는 향후에 투자할 예정입니다. 또한, 구글은 앤쓰로픽과의 클라우드 서비스 사용 계약도 체결하여, 앤쓰로픽은 4년간 30억 달러 규모의 구글 클라우드 서비스를 사용하게 됩니다【4:0†source】.


In [12]:
# 추가 질문
print(assistant.invoke("이전의 답변을 보고서 양식으로 작성해 주세요"))

## 보고서: 구글의 앤쓰로픽 투자 계획

### 개요
이 보고서는 구글이 인공지능 연구기업 앤쓰로픽(Anthropic)에 대한 투자 계획을 발표한 내용을 다룹니다. 해당 투자는 구글의 AI 및 클라우드 서비스 분야에서의 전략적 협력을 강화하며, 양사간의 긴밀한 협력 관계를 형성하는데 목적이 있습니다.

### 투자 합의 내용
구글은 앤쓰로픽에 최대 20억 달러를 투자하기로 합의했습니다. 투자 계획은 두 단계로 진행됩니다:
1. **1단계**: 구글은 초기 투자로 5억 달러를 앤쓰로픽에 투자하였습니다.
2. **2단계**: 추가적으로 구글은 향후 15억 달러를 앤쓰로픽에 투자할 계획입니다.

### 클라우드 서비스 계약
투자 외에도 구글과 앤쓰로픽은 클라우드 서비스 사용 계약을 체결했습니다. 앤쓰로픽은 앞으로 4년간 구글 클라우드(Google Cloud) 서비스를 사용하며, 이 계약의 규모는 30억 달러에 달합니다. 이를 통해 앤쓰로픽은 구글의 최첨단 클라우드 인프라를 활용할 수 있게 되며, 구글 클라우드는 안정적인 수익을 확보하게 됩니다.

### 기대 효과와 전망
이번 투자는 구글이 인공지능 분야에서 지배적인 위치를 더욱 공고히 하는 계기가 될 것으로 보입니다. 뿐만 아니라, 구글 클라우드 서비스를 통해 앤쓰로픽의 AI 연구 및 개발이 가속화될 전망입니다. 양사 간의 협력은 기술 혁신과 시장 경쟁력 강화에 크게 기여할 것으로 기대됩니다.

### 결론
구글의 앤쓰로픽에 대한 투자와 클라우드 서비스 계약 체결은 AI와 클라우드 컴퓨팅 분야에서의 긴밀한 협력으로 이어질 것입니다. 이는 양사 모두에게 중요한 전략적 결정으로, 향후 기술 발전과 시장에서의 유리한 입지 확보에 기여할 것으로 보입니다.

출처: 구글 2023년 10월 27일 발표【4:0†source】.


In [13]:
# 대화 목록 조회
assistant.list_chat_history()

[{'content': '구글이 앤쓰로픽에 투자 계획을 발표했습니다. 이에 대해 알려줘', 'role': 'user'},
 {'content': '구글은 2023년 10월 27일에 앤쓰로픽에 최대 20억 달러를 투자하기로 합의했습니다. 이 중 5억 달러를 우선 투자하였으며, 나머지 15억 달러는 향후에 투자할 예정입니다. 또한, 구글은 앤쓰로픽과의 클라우드 서비스 사용 계약도 체결하여, 앤쓰로픽은 4년간 30억 달러 규모의 구글 클라우드 서비스를 사용하게 됩니다【4:0†source】.',
  'role': 'assistant'},
 {'content': '이전의 답변을 보고서 양식으로 작성해 주세요', 'role': 'user'},
 {'content': '## 보고서: 구글의 앤쓰로픽 투자 계획\n\n### 개요\n이 보고서는 구글이 인공지능 연구기업 앤쓰로픽(Anthropic)에 대한 투자 계획을 발표한 내용을 다룹니다. 해당 투자는 구글의 AI 및 클라우드 서비스 분야에서의 전략적 협력을 강화하며, 양사간의 긴밀한 협력 관계를 형성하는데 목적이 있습니다.\n\n### 투자 합의 내용\n구글은 앤쓰로픽에 최대 20억 달러를 투자하기로 합의했습니다. 투자 계획은 두 단계로 진행됩니다:\n1. **1단계**: 구글은 초기 투자로 5억 달러를 앤쓰로픽에 투자하였습니다.\n2. **2단계**: 추가적으로 구글은 향후 15억 달러를 앤쓰로픽에 투자할 계획입니다.\n\n### 클라우드 서비스 계약\n투자 외에도 구글과 앤쓰로픽은 클라우드 서비스 사용 계약을 체결했습니다. 앤쓰로픽은 앞으로 4년간 구글 클라우드(Google Cloud) 서비스를 사용하며, 이 계약의 규모는 30억 달러에 달합니다. 이를 통해 앤쓰로픽은 구글의 최첨단 클라우드 인프라를 활용할 수 있게 되며, 구글 클라우드는 안정적인 수익을 확보하게 됩니다.\n\n### 기대 효과와 전망\n이번 투자는 구글이 인공지능 분야에서 지배적인 위치를 더욱 공고히 하는 계기가 될 것으로 보입니다. 뿐만 아니라, 구